# Introduction 

Author: Yuzhu Zhang (yuzhu.zhang@deliveryhero.com)

In [1]:
# load packages
#from autoplotter import run_app # Importing the autoplotter for GUI Based EDA
from datetime import datetime, timedelta, time, date
from scipy import stats
import json
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.io import gbq
from google.cloud import bigquery
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import scipy.stats
#import geopandas
from shapely import wkt
import scipy
from scipy import signal
%matplotlib inline
#from google.colab import drive # to allow colab save file in my drive
#drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
cwd=os.getcwd()
cwd
%unload_ext google.colab.data_table

Authenticated
The google.colab.data_table extension is not loaded.


In [2]:
# define a few things (project id, start date etc.)
project_id = "fulfillment-dwh-production"
client = bigquery.Client(project = project_id)
entity_id = 'FP_HK'
country_code = 'hk'
start_date = '2020-08-21'
end_date = '2020-09-14'
time_zone = "Asia/Hong_Kong" # to check delay data
zone_id = "52,70" # 3 = Far East, If you need two zones like zone 9 and zone 35, then put "9,35" behind "="
variant = "'Variation1','Variation2','Variation3', 'Original','Control'"
#zone_name = "'Novena','Yishun'"
#%cd "/content/drive/Shared drives/Global Pricing/2 - Entities/APAC/Singapore/7. DPS testing/2. DPS_AB_Testing_20200902"
#%pwd
#%ls

In [4]:
# trunc time function
def ceil_dt(dt, delta):
    return dt + (datetime.min - dt) % delta

# formatting
def color_negative_red(val):
    color = 'green' if val == True else 'black'
    return f'color: {color}'

# 2. Methodology

## A/B testing

We will check the statistical/substantial significance level of the key metrics.
To check the statistical significance level, we will group the data into hours and make paired comparison.

What is Wilcoxon signed-rank test?
The Wilcoxon signed-rank test is a non-parametric statistical hypothesis test used to compare two related samples, matched samples, or repeated measurements on a single sample to assess whether their population mean ranks differ (i.e. it is a paired difference test). It can be used as an alternative to the paired Student's t-test (also known as "t-test for matched pairs" or "t-test for dependent samples") when the distribution of the difference between two samples' means cannot be assumed to be normally distributed.[1] A Wilcoxon signed-rank test is a nonparametric test that can be used to determine whether two dependent samples were selected from populations having the same distribution.

As we could not assume that the distribution will be normal, we will use wilcoxon signed-rank test to check the significance


Ressource: 

*   https://medium.com/@kangeugine/hypothesis-test-21795f788f7d
*   https://stackoverflow.com/questions/15984221/how-to-perform-two-sample-one-tailed-t-test-with-numpy-scipy






# 3. A/B Test Analysis

## Sanity Check

### a. Invariant: user amount in control and variantion

In [ ]:
# Pre-condition check
query_user = """
WITH city_data AS (
  SELECT p.entity_id
    , country_code
    , ci.name AS city_name
    , ci.id AS city_id
    , zo.shape AS zone_shape 
    , zo.name AS zone_name
    , zo.id AS zone_id
  FROM cl.countries co
  LEFT JOIN UNNEST(co.platforms) p
  LEFT JOIN UNNEST(co.cities) ci
  LEFT JOIN UNNEST(ci.zones) zo
  WHERE entity_id = \"""" + entity_id + """\"
  and zo.id in (""" + zone_id + """)
)
 
SELECT  
zone_name
, customer.variant
, count(distinct customer.id) as customer_count
, count(distinct customer.user_id) as user_count
FROM `fulfillment-dwh-production.cl.dynamic_pricing_user_sessions` s
left join city_data cd ON s.entity_id = cd.entity_id
WHERE cast(DATETIME(created_at, "Asia/Singapore") as date) between \"""" + start_date + """\" and \"""" + end_date + """\"
and created_date >= \"""" + start_date + """\" 
and s.entity_id = \"""" + entity_id + """\"
--and customer.variant in ("Variation1","Variation2","Variation3", "Original","Control")
and ST_CONTAINS(cd.zone_shape, customer.location) IS TRUE
group by 1,2

    """

zone_whole = client.query(query_user).to_dataframe()
zone_whole.head()

,zone_name,variant,customer_count,user_count
0,Tin shui wai walker,Control,19499,22847
1,Tin shui wai walker,Variation1,19721,22869
2,Tin shui wai walker,Original,15266,16777
3,Tin shui wai rider,Variation1,23002,26878
4,Tin shui wai rider,Original,18760,20890


In [ ]:
zone_whole = zone_whole[~zone_whole.zone_name.str.contains("Dmart",na=False)]
zone_names = ['Amk', 'Bedok', 'Bukit timah', 'Bukitpanjang', 'Downtown','Far_east', 'Geylang', 'Jurong east', 'Jurongwest', 'Novena','Sengkang', 'Serangoon', 'Woodlands', 'Yishun']
zwhole_test = zone_whole.loc[zone_whole["variant"]!= "Original"]
zwhole_test
fig = px.bar(zwhole_test, x='variant', y='user_count', text = "user_count", facet_col="zone_name", facet_col_wrap=5,
             category_orders = {#"zone_name":zone_names, 
                                 "variant": ["Control", "Variation1", "Variation2", "Variation3"]})
fig.show()

## Deep Dive
We look at different metrics:

Economics Metrics
*   Orders
*   Delivery fee
*   Basket value
*   Revenue/Profit

Logistics Metrics 

*   Travel Time


In [5]:
query1 = """
WITH 

costs as (
  select
    p.entity_id,
    l.platform_order_code platform_order_code,
    sum(p.delivery_costs) delivery_costs,
    sum(p.delivery_costs_eur) delivery_costs_eur
  from 
    (select
      entity_id,
      country_code,
      created_date,
      order_id,
      delivery_costs,
      delivery_costs_eur,
      row_number() over(partition by entity_id, order_id order by created_date desc) as rank
    from cl.utr_timings) p
  left join cl.orders l on p.order_id = l.order_id and p.country_code = l.country_code
  where p.entity_id = \"""" + entity_id + """\"
    and rank = 1
  group by 1,2)

select
zone_name
, zone_id
, vertical_type
, operating_system
, cast(DATETIME(created_at, o.timezone) as date) as local_date
, DATETIME(created_at, o.timezone) as local_time
, vendor_code
, platform_order_code_ga
, variant
, dps_test_zone
, dps_customer_tag
, gmv_local
, gfv_local
, cast(value.commission_local as float64) commission_local
, cast(value.commission_eur as float64) commission_eur
, delivery_fee_local
, delivery_fee_eur
, delivery_fee_local_accounting
, travel_time_distance_km
, mean_delay
, travel_time
, to_customer_time
, to_vendor_time
, delivery_distance
, actual_delivery_time
, order_delay_mins
, delivery_costs
, delivery_costs_eur
from cl._dps_sessions_mapped_to_orders o
left join costs cos on cos.entity_id =o.entity_id and cos.platform_order_code = o.platform_order_code_ga
left join `fulfillment-dwh-production.cl_central_dwh.orders` ceno on o.entity_id = ceno.global_entity_id and o.platform_order_code_ga = ceno.order_id
where o.entity_id = \"""" + entity_id + """\"
and zone_id in (""" + zone_id + """)
and variant in ("Control","Variation1","Variation2","Variation3")
and cast(DATETIME(created_at, o.timezone) as date) between \"""" + start_date + """\"  and \"""" + end_date + """\"
and platform_order_code_ga is not null
    """

df = client.query(query1).to_dataframe()

In [10]:
# define time block
df["local_hour"] = df.local_time.dt.hour
df["hour_block"] = df["local_hour"]
df.loc[df.local_hour <= 9, 'hour_block'] = 9
df.loc[df.local_hour >=21, 'hour_block'] = 21
df["rev"] = df["commission_local"] + df["delivery_fee_local"]
df["profit"] = df["rev"] - df["delivery_costs"]
df["profit_eur"] = df["commission_eur"] + df["delivery_fee_eur"] - df["delivery_costs_eur"]
#df["new_time"] = ceil_dt((df['local_time'].dt.to_pydatetime()), timedelta(minutes=30))
#df["new_time"] = df['new_time'].dt.time
#df.loc[df.new_time <= time(hour=8, minute=0, second=0), 'new_time'] = time(hour=8, minute=0, second=0)

# Create tier
df["tier"] = df["travel_time"]

conditions = [
    (df['tier'] <= 1.86),
    (df['tier'] > 1.86) & (df['tier'] <= 2.53),
    (df['tier'] > 2.53) & (df['tier'] < 3.5),
    (df['tier'] >= 3.5) & (df['tier'] < 5.66),
    (df['tier'] >= 5.66)
    ]
values = ["Control 5 & Test: 5", "Control 5 & Test 10", "Control 15 & Test 10","Control 15 & Test 15", "DF>15"]
df["tier"] = np.select(conditions,values)

df = df.fillna(0)
df = df.drop_duplicates()
df["rev_sum"] = df["rev"]
df
#keep an extra copy
df_copy = df.copy()
#delete the date which dps do not have control or variation
date_list = [date(year=2020, month=8, day=31), date(year=2020, month=9, day=1), date(year=2020, month=9, day=2), date(year=2020, month=9, day=3)]
df = df.loc[~df['local_date'].isin(date_list)]
df.head()

,zone_name,zone_id,vertical_type,operating_system,local_date,local_time,vendor_code,platform_order_code_ga,variant,dps_test_zone,dps_customer_tag,gmv_local,gfv_local,commission_local,commission_eur,delivery_fee_local,delivery_fee_eur,delivery_fee_local_accounting,travel_time_distance_km,mean_delay,travel_time,to_customer_time,to_vendor_time,delivery_distance,actual_delivery_time,order_delay_mins,delivery_costs,delivery_costs_eur,local_hour,hour_block,rev,profit,tier,rev_sum,profit_eur
0,Tin shui wai rider,70,restaurants,Android,2020-09-10,2020-09-10 10:37:21.883916,v3gc,v3gc-m1zu,Variation1,New Price Point,0,130.0,105.0,26.46,2.89,25.0,2.732697,25.0,1.431475,10.721766,10.67,0.00,6.08,1.520,21.27,0.00,29.396,3.207,10,10,51.46,22.064,DF>15,51.46,2.415697
1,Tin shui wai rider,70,restaurants,Web,2020-09-10,2020-09-10 20:26:03.598387,l0ik,l0ik-9hsc,Variation1,New Price Point,0,83.0,58.0,14.50,1.58,25.0,2.732697,25.0,1.104782,8.383840,8.24,0.00,3.92,1.274,19.80,0.00,41.100,4.484,20,20,39.50,-1.600,DF>15,39.50,-0.171303
2,Tin shui wai rider,70,restaurants,iOS,2020-09-10,2020-09-10 13:01:36.461753,g3rw,g3rw-xlan,Control,New Price Point,0,132.0,107.0,24.07,2.63,25.0,2.732697,25.0,1.542998,8.507118,11.50,27.02,0.30,2.179,44.85,10.97,33.089,3.610,13,13,49.07,15.981,DF>15,49.07,1.752697
3,Tin shui wai walker,52,restaurants,iOS,2020-09-10,2020-09-10 18:10:08.042450,x1lw,x1lw-35rv,Variation1,New Price Point,0,249.0,239.0,53.77,5.88,10.0,1.093079,10.0,0.315482,6.799336,2.35,0.00,2.03,0.355,25.45,0.00,50.072,5.463,18,18,63.77,13.698,Control 5 & Test 10,63.77,1.510079
4,Tin shui wai walker,52,restaurants,iOS,2020-09-10,2020-09-10 12:25:57.857629,v7ui,v7ui-og9h,Variation1,New Price Point,0,69.0,59.0,14.75,1.61,10.0,1.093079,10.0,0.304396,9.312095,2.27,0.00,0.00,0.409,7.50,0.00,21.828,2.382,12,12,24.75,2.922,Control 5 & Test 10,24.75,0.321079


In [19]:
check = df_copy.pivot_table(values = ["delivery_fee_eur","profit_eur"], 
                            #index=["local_date", "hour_block"],
                        columns = "variant",
                       aggfunc ={"delivery_fee_eur":"mean","profit_eur":"sum"}).reset_index()
check

variant,index,Control,Variation1
0,delivery_fee_eur,1.449502,1.470352
1,profit_eur,7792.565692,9174.138974


### User frequency

#### User amount

In [ ]:
# Pre-condition check
query_user = """
WITH city_data AS (
  SELECT p.entity_id
    , country_code
    , ci.name AS city_name
    , ci.id AS city_id
    , zo.shape AS zone_shape 
    , zo.name AS zone_name
    , zo.id AS zone_id
  FROM cl.countries co
  LEFT JOIN UNNEST(co.platforms) p
  LEFT JOIN UNNEST(co.cities) ci
  LEFT JOIN UNNEST(ci.zones) zo
  WHERE entity_id = \"""" + entity_id + """\"
  and zo.id in (70)
)
 
SELECT  
cast(DATETIME(created_at, "Asia/Hong_Kong") as date) as local_date
--extract(hour from DATETIME(created_at, "Asia/Hong_Kong")) as hour
, zone_id
, zone_name
, customer.variant
, count(distinct customer.id) as customer_count
, count(distinct customer.user_id) as user_count
, count(distinct customer.session.id) as session_count
FROM `fulfillment-dwh-production.cl.dynamic_pricing_user_sessions` s
left join city_data cd ON s.entity_id = cd.entity_id
WHERE cast(DATETIME(created_at, "Asia/Singapore") as date) between \"""" + start_date + """\" and \"""" + end_date + """\"
and created_date >= \"""" + start_date + """\" 
and s.entity_id = \"""" + entity_id + """\"
and customer.variant in ("Variation1","Variation2","Variation3", "Control")
and ST_CONTAINS(cd.zone_shape, customer.location) IS TRUE
group by 1,2,3,4

    """

user = client.query(query_user).to_dataframe()
user.head()

,local_date,zone_id,zone_name,variant,customer_count,user_count,session_count
0,2020-09-11,70,Tin shui wai rider,Variation1,3480,3768,6621
1,2020-09-13,70,Tin shui wai rider,Control,4085,4443,8449
2,2020-08-26,70,Tin shui wai rider,Control,3717,4055,7574
3,2020-09-08,70,Tin shui wai rider,Control,3345,3619,6608
4,2020-08-28,70,Tin shui wai rider,Variation1,3759,4056,8044


In [ ]:
user_group = user.pivot_table("user_count", index=["zone_name","local_date"], columns ="variant", aggfunc = "sum").reset_index()
user_group["delta_v1c"]=user_group["Variation1"]/user_group["Control"]-1
#user_group['delta_v1c'] = user_group['delta_v1c'].astype(float).map(lambda n: '{:.2%}'.format(n))

user2 = user_group.copy()
user2.drop(["Control","Variation1"], axis=1, inplace=True)
user2 = pd.melt(user2, id_vars=['local_date',"zone_name"], value_vars=['delta_v1c'])
user2.rename(columns={'variant': 'Variation vs Control', 'value': 'delta'}, inplace=True)

user_group = user.pivot_table("user_count", index=["zone_name","local_date"], columns ="variant", aggfunc = "sum").reset_index()
user_group


fig = px.line(user2, x="local_date", y="delta", color = "Variation vs Control", facet_col="zone_name", facet_col_wrap=5,
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=800, width=1600,
              category_orders = {"variant": ["Control", "Variation1", "Variation2", "Variation3"]},
              title="User amount delta in all Tin shui wai across testing period")
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(showticklabels=True)

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''

fig.show()

In [ ]:
print("mean difference is",round(user_group['Variation1'].mean()/user_group['Control'].mean()-1,4),"p-value is",stats.wilcoxon(user_group['Variation1'], user_group['Control'])[1]/2)

mean difference is 0.0234 p-value is 1.5441973677250172e-05


In [ ]:
user_group

variant,zone_name,local_date,Control,Variation1
0,Tin shui wai rider,2020-08-21,4097,4188
1,Tin shui wai rider,2020-08-22,5064,5158
2,Tin shui wai rider,2020-08-23,5381,5525
3,Tin shui wai rider,2020-08-24,3976,4110
4,Tin shui wai rider,2020-08-25,4321,4361
5,Tin shui wai rider,2020-08-26,4055,4021
6,Tin shui wai rider,2020-08-27,3817,3941
7,Tin shui wai rider,2020-08-28,3947,4056
8,Tin shui wai rider,2020-08-29,4351,4606
9,Tin shui wai rider,2020-08-30,4747,4804


#### Session amount

In [ ]:
user_group = user.pivot_table("session_count", index=["zone_name","local_date"], columns ="variant", aggfunc = "sum").reset_index()
#user_group = user_group.loc[user_group["local_date"] < date(year=2020, month=11, day=2)]
user_group["delta_v1c"]=user_group["Variation1"]/user_group["Control"]-1
#user_group['delta_v1c'] = user_group['delta_v1c'].astype(float).map(lambda n: '{:.2%}'.format(n))
#user_group['delta_v2c'] = user_group['delta_v2c'].astype(float).map(lambda n: '{:.2%}'.format(n))
#user_group['delta_v3c'] = user_group['delta_v3c'].astype(float).map(lambda n: '{:.2%}'.format(n))
user2 = user_group.copy()
user2.drop(["Control","Variation1"], axis=1, inplace=True)
user2 = pd.melt(user2, id_vars=['local_date',"zone_name"], value_vars=['delta_v1c'])
user2.rename(columns={'variant': 'Variation vs Control', 'value': 'delta'}, inplace=True)

user_group = user.pivot_table("session_count", index=["zone_name","local_date"], columns ="variant", aggfunc = "sum").reset_index()
user_group


fig = px.line(user2, x="local_date", y="delta", color = "Variation vs Control", facet_col="zone_name", facet_col_wrap=5,
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=800, width=1600,
              category_orders = {"zone_name":['Amk', 'Bedok', 'Bukit timah', 'Bukitpanjang', 'Downtown','Far_east', 'Geylang', 'Jurong east', 'Jurongwest', 'Novena','Sengkang', 'Serangoon', 'Woodlands', 'Yishun'], 
                                 "variant": ["Control", "Variation1", "Variation2", "Variation3"]},
              title="Session Delta in Tin Shui Wai across testing period")
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(showticklabels=True)

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''

fig.show()

In [ ]:
print("mean difference is",round(user_group['Variation1'].mean()/user_group['Control'].mean()-1,4),"p-value is",stats.wilcoxon(user_group['Variation1'], user_group['Control'])[1]/2)

mean difference is 0.0298 p-value is 0.00026285283308807834


### Orders

#### Aggregated level

In [ ]:
df_agg = df.copy()
df_agg["rev_sum"] = df_agg["rev"]
df_agg = df_agg.pivot_table(values = ["platform_order_code_ga", "delivery_fee_local","gfv_local","rev","rev_sum","travel_time",], index=["local_date", "variant"], 
                       aggfunc ={"platform_order_code_ga": lambda x: len(x.unique()),"delivery_fee_local":"mean","gfv_local":"mean","rev":"mean","travel_time":"mean","rev_sum":"sum" }).reset_index()

In [ ]:
fig = px.line(df_agg, x="local_date", y="platform_order_code_ga", color = "variant", 
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=600, width=1200,
              title="Aggregated order amount across testing period")

# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Order Amount", showgrid=False)

fig.show()

In [12]:
df_sig = df.pivot_table(values = ["platform_order_code_ga", "delivery_fee_local","gfv_local","rev","rev_sum","travel_time","profit"], index=["local_date","hour_block"], columns = ["variant"],
                       aggfunc ={"platform_order_code_ga": "count","delivery_fee_local":"mean","gfv_local":"mean","rev":"mean","rev_sum":"sum","travel_time":"mean","profit":"sum"}).reset_index()
df_sig.rename(columns={'dps_delivery_fee': 'dps_df', 'platform_order_code_ga': 'order'}, inplace=True)
#df_data.columns = df_data.columns.get_level_values(1)
df_sig.columns = ['_'.join(col).strip() for col in df_sig.columns.values]
df_sig.rename(columns={'local_date_': 'local_date', 'hour_block_': 'hour_block'}, inplace=True)

In [ ]:
df_sig.columns

Index(['local_date', 'hour_block', 'delivery_fee_local_Control',
       'delivery_fee_local_Variation1', 'gfv_local_Control',
       'gfv_local_Variation1', 'order_Control', 'order_Variation1',
       'profit_Control', 'profit_Variation1', 'rev_Control', 'rev_Variation1',
       'rev_sum_Control', 'rev_sum_Variation1', 'travel_time_Control',
       'travel_time_Variation1'],
      dtype='object')

In [ ]:
print("mean difference is",round(df_sig['order_Variation1'].mean()/df_sig['order_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['order_Variation1'], df_sig['order_Control'])[1]/2)

mean difference is 0.0353 p-value is 1.540847105440353e-05


In [ ]:
print("mean difference is",round(df_sig['profit_Variation1'].mean()/df_sig['profit_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['profit_Variation1'], df_sig['profit_Control'])[1]/2)

mean difference is 0.1775 p-value is 0.015204068260880258


In [ ]:
print("mean difference is",round(df_sig['travel_time_Variation1'].mean()/df_sig['travel_time_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['travel_time_Variation1'], df_sig['travel_time_Control'])[1]/2)

mean difference is 0.0126 p-value is 0.01689326704929512


#### Tier level

In [ ]:
# order data analysis
df_tier = df.pivot_table(values = ["platform_order_code_ga", "delivery_fee_local","gfv_local","rev","travel_time", "rev_sum"], index=["local_date", "tier","variant"], 
                       aggfunc ={"platform_order_code_ga": lambda x: len(x.unique()),"delivery_fee_local":"mean","gfv_local":"mean","rev":"mean","travel_time":"mean", "rev_sum":"sum" }).reset_index()
df_tier

,local_date,tier,variant,delivery_fee_local,gfv_local,platform_order_code_ga,rev,rev_sum,travel_time
0,2020-08-21,Control 15 & Test 10,Control,14.357143,137.750000,140,46.330810,6486.313429,3.023429
1,2020-08-21,Control 15 & Test 10,Variation1,9.845361,133.755155,194,42.221311,8190.934389,3.003763
2,2020-08-21,Control 15 & Test 15,Control,14.715719,144.917391,299,50.471329,15090.927500,4.494482
3,2020-08-21,Control 15 & Test 15,Variation1,14.683099,141.255282,284,50.240025,14268.167100,4.530246
4,2020-08-21,Control 5 & Test 10,Control,5.181818,111.978788,165,33.091945,5460.171000,2.170727
...,...,...,...,...,...,...,...,...,...
205,2020-09-14,Control 5 & Test 10,Variation1,9.850299,107.697605,167,37.056386,6188.416500,2.190060
206,2020-09-14,Control 5 & Test: 5,Control,5.000000,96.453875,271,30.221866,8190.125698,1.279299
207,2020-09-14,Control 5 & Test: 5,Variation1,4.957627,94.773305,236,29.336392,6923.388500,1.291780
208,2020-09-14,DF>15,Control,21.039886,141.297151,351,56.009067,19659.182600,8.463932


In [ ]:
fig = px.line(df_tier, x="local_date", y="platform_order_code_ga", color = "variant", facet_col="tier", facet_col_wrap=5,
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=800, width=1600,
              category_orders = {"tier":["Control 5 & Test: 5", "Control 5 & Test 10", "Control 15 & Test 10","Control 15 & Test 15", "DF>15"], 
                                 "variant": ["Control", "Variation1", "Variation2", "Variation3"]},
              title="Order amount in different tier across testing period")
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(showticklabels=True)

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''

fig.show()

In [ ]:
# Prepare for the data
df_data = df[df["variant"] != 'Original']
df_data = pd.pivot_table(df_data, values=["dps_delivery_fee","platform_order_code_ga", "gfv_local", "rev","travel_time"], index=["local_date", "hour_block", "tier"],columns= "variant", 
                         aggfunc = {'dps_delivery_fee' : 'sum', 'platform_order_code_ga' : 'count', 'gfv_local' : 'sum', "rev":"sum", "travel_time":"sum"}).reset_index()
df_data.rename(columns={'dps_delivery_fee': 'sum_df', 'platform_order_code_ga': 'order'}, inplace=True)
#df_data.columns = df_data.columns.get_level_values(1)
df_data.columns = ['_'.join(col).strip() for col in df_data.columns.values]
df_data.rename(columns={'local_date_': 'local_date', 'hour_block_': 'hour_block','tier_': 'tier','zone_name_':'zone_name'}, inplace=True)
df_data.columns = map(str.lower, df_data.columns)
df_data["avg_df_control"] = df_data["sum_df_control"]/df_data["order_control"]
df_data["avg_df_variation1"] = df_data["sum_df_variation1"]/df_data["order_variation1"]
df_data["avg_gfv_control"] = df_data["gfv_local_control"]/df_data["order_control"]
df_data["avg_gfv_variation1"] = df_data["gfv_local_variation1"]/df_data["order_variation1"]
df_data["avg_rev_control"] = df_data["rev_control"]/df_data["order_control"]
df_data["avg_rev_variation1"] = df_data["rev_variation1"]/df_data["order_variation1"]
df_data = df_data.fillna(0)
df_data.head()

,local_date,hour_block,tier,sum_df_control,sum_df_variation1,gfv_local_control,gfv_local_variation1,order_control,order_variation1,rev_control,rev_variation1,travel_time_control,travel_time_variation1,avg_df_control,avg_df_variation1,avg_gfv_control,avg_gfv_variation1,avg_rev_control,avg_rev_variation1
0,2020-08-21,9,Control 15 & Test 10,125.714286,185.0,1264.5,1925.0,10,19,410.114286,695.690,31.46,57.79,12.571429,9.736842,126.450000,101.315789,41.011429,36.615263
1,2020-08-21,9,Control 15 & Test 15,210.000000,205.0,1357.0,1700.0,14,17,540.157000,631.724,57.54,74.67,15.000000,12.058824,96.928571,100.000000,38.582643,37.160235
2,2020-08-21,9,Control 5 & Test 10,70.000000,105.0,1007.0,1071.5,14,13,303.070000,418.758,30.33,28.31,5.000000,8.076923,71.928571,82.423077,21.647857,32.212154
3,2020-08-21,9,Control 5 & Test: 5,100.000000,160.0,1619.0,2558.5,21,32,530.988000,844.480,29.52,42.52,4.761905,5.000000,77.095238,79.953125,25.285143,26.390000
4,2020-08-21,9,DF>15,465.000000,455.0,2517.0,3052.0,25,26,1041.286000,1159.119,209.71,224.44,18.600000,17.500000,100.680000,117.384615,41.651440,44.581500


In [ ]:
#zone_names = ['Amk', 'Bedok', 'Bukit timah', 'Bukitpanjang', 'Downtown','Far_east', 'Geylang', 'Jurong east', 'Jurongwest', 'Novena','Sengkang', 'Serangoon', 'Woodlands', 'Yishun']
zone_names = (df["tier"].unique()).tolist()

In [ ]:
# create empty array list
rows_list = []

# loop the data for each zone and each stage
for zone_name in zone_names:
    zone_data = df_data[df_data["tier"] == zone_name] #filter relevant zone and stage for each loop
    zone_data = zone_data.fillna(0)
    mean_control = round(zone_data["order_control"].mean(),2)
    mean_variation1 = round(zone_data["order_variation1"].mean(),2)
    std_control = round(zone_data["order_control"].std(),2)
    std_variation1 = round(zone_data["order_variation1"].std(),2)
    cor_control_variation1 = round(zone_data["order_control"].corr(zone_data["order_variation1"]),2)
    sample_size_control = zone_data["order_control"].sum()
    sample_size_variation1 = zone_data["order_variation1"].sum()
    #sample_size_total = sample_size_control + sample_size_variation1
    sig_control_variation1_oneside = round(stats.wilcoxon(zone_data['order_variation1'], zone_data['order_control']) [1]/2,2)
    rows_list.append([zone_name, mean_control, mean_variation1,
                      std_control, std_variation1,
                      cor_control_variation1, 
                      sample_size_control, sample_size_variation1,  
                      sig_control_variation1_oneside])

# load data into dataframe
result = pd.DataFrame(rows_list, columns=['zone_name','mean_control','mean_variation1',
                                          'std_control', 'std_variation1',
                                          'correlation (cV1)',  
                                          'size_control', 'size_variation1', 
                                          'p-value(v1 vs control)'])
# add two columns to data frame
#result["significance"] = result["p-value"]<=0.05
result["delta(v1 vs control)"] = round(result["mean_variation1"]/result["mean_control"]-1,2)
result.rename(columns={"zone_name":"tier"}, inplace=True)

result.sort_values("tier")

,tier,mean_control,mean_variation1,std_control,std_variation1,correlation (cV1),size_control,size_variation1,p-value(v1 vs control),delta(v1 vs control)
3,Control 15 & Test 10,11.64,14.49,5.83,6.95,0.67,3179,3957,0.00,0.24
1,Control 15 & Test 15,23.48,25.07,11.36,11.69,0.82,6410,6844,0.00,0.07
0,Control 5 & Test 10,14.15,12.07,6.82,5.56,0.69,3862,3294,0.00,-0.15
4,Control 5 & Test: 5,22.18,22.12,9.85,8.70,0.75,6054,6038,0.37,-0.00
2,DF>15,32.86,34.29,16.80,16.51,0.89,8970,9360,0.00,0.04


### Delivery Fee

#### Aggregated level

In [ ]:
fig = px.line(df_agg, x="local_date", y="delivery_fee_local", color = "variant", 
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=600, width=1200,
              title="Avg. DF across testing period")

# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Delivery Fee", showgrid=False)

fig.show()

In [ ]:
fig = px.line(df_agg, x="local_date", y="dps_delivery_fee", color = "variant", 
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=600, width=1200,
              title="Avg. DF across testing period")

# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Delivery Fee", showgrid=False)

fig.show()

In [ ]:
print("mean difference is",round(df_sig['delivery_fee_local_Variation1'].mean()/df_sig['delivery_fee_local_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['delivery_fee_local_Variation1'], df_sig['delivery_fee_local_Control'])[1]/2)

mean difference is 0.0154 p-value is 0.0009215581729197336


In [ ]:
print("mean difference is",round(df_sig['dps_df_Variation1'].mean()/df_sig['dps_df_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['dps_df_Variation1'], df_sig['dps_df_Control'])[1]/2)

mean difference is 0.0157 p-value is 0.0009311735071108997


#### Tier level

In [ ]:
fig = px.line(df_tier, x="local_date", y="delivery_fee_local", color = "variant", facet_col="tier", facet_col_wrap=5,
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=800, width=1600,
              category_orders = {"zone_name":['Amk', 'Bedok', 'Bukit timah', 'Bukitpanjang', 'Downtown','Far_east', 'Geylang', 'Jurong east', 'Jurongwest', 'Novena','Sengkang', 'Serangoon', 'Woodlands', 'Yishun'], 
                                 "variant": ["Control", "Variation1", "Variation2", "Variation3"]},
              title="User amount in all SG Zones across testing period")
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(showticklabels=True)

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''

fig.show()

In [ ]:
# create empty array list
rows_list = []

# loop the data for each zone and each stage
for zone_name in zone_names:
    zone_data = df_data[df_data["tier"] == zone_name] #filter relevant zone and stage for each loop
    zone_data = zone_data.fillna(0)
    mean_control = round(zone_data["sum_df_control"].sum()/zone_data["order_control"].sum(),2)
    mean_variation1 = round(zone_data["sum_df_variation1"].sum()/zone_data["order_variation1"].sum(),2)
    std_control = round(zone_data["avg_df_control"].std(),2)
    std_variation1 = round(zone_data["avg_df_variation1"].std(),2)
    cor_control_variation1 = round(zone_data["avg_df_control"].corr(zone_data["avg_df_variation1"]),2)
    sample_size_control = zone_data["order_control"].sum()
    sample_size_variation1 = zone_data["order_variation1"].sum()
    #sample_size_total = sample_size_control + sample_size_variation1
    sig_control_variation1_oneside = round(stats.wilcoxon(zone_data['avg_df_variation1'], zone_data['avg_df_control']) [1]/2,2)
    rows_list.append([zone_name, mean_control, mean_variation1,
                      std_control, std_variation1, 
                      cor_control_variation1,
                      sample_size_control, sample_size_variation1, 
                      sig_control_variation1_oneside])

# load data into dataframe
result = pd.DataFrame(rows_list, columns=['zone_name','mean_control','mean_variation1',
                                          'std_control', 'std_variation1',
                                          'correlation (cV1)', 
                                          'size_control', 'size_variation1', 
                                          'p-value(v1 vs control)'])
# add two columns to data frame
#result["significance"] = result["p-value"]<=0.05
result["delta(v1 vs control)"] = round(result["mean_variation1"]/result["mean_control"]-1,2)
result.rename(columns={"zone_name":"tier"}, inplace=True)

result

,tier,mean_control,mean_variation1,std_control,std_variation1,correlation (cV1),size_control,size_variation1,p-value(v1 vs control),delta(v1 vs control)
0,Control 5 & Test 10,4.97,9.58,0.61,0.82,0.26,3862,3294,0.00,0.93
1,Control 15 & Test 15,14.35,14.24,0.91,0.79,0.21,6410,6844,0.07,-0.01
2,DF>15,20.68,20.64,1.25,1.19,0.15,8970,9360,0.12,-0.00
3,Control 15 & Test 10,14.09,9.61,1.22,0.72,0.07,3179,3957,0.00,-0.32
4,Control 5 & Test: 5,4.90,4.93,0.30,0.27,0.02,6054,6038,0.01,0.01


### 3. Basket Value

#### Aggregated level

In [ ]:
fig = px.line(df_agg, x="local_date", y="gfv_local", color = "variant", 
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=600, width=1200,
              title="Avg. GFV across testing period")

# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Gross Food Value", showgrid=False)

fig.show()

Significance Check

In [ ]:
print("mean difference is",round(df_sig['gfv_local_Variation1'].mean()/df_sig['gfv_local_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['gfv_local_Variation1'], df_sig['gfv_local_Control'])[1]/2)

mean difference is -0.0004 p-value is 0.22657610986693272


In [ ]:
bas = df[["platform_order_code_ga","gfv_local","variant"]].copy()
bas["gfv_round"]=bas["gfv_local"].round(1)
bas = bas.pivot_table("platform_order_code_ga", index=["gfv_round"],columns="variant",aggfunc ="count").reset_index()
bas = bas.fillna(0)
bas["pp_control"] = bas["Control"]/bas["Control"].sum()
bas["pp_variation1"] = bas["Variation1"]/bas["Variation1"].sum()
bas

variant,gfv_round,Control,Variation1,pp_control,pp_variation1
0,11.0,1.0,0.0,0.000035,0.000000
1,12.0,1.0,0.0,0.000035,0.000000
2,17.0,1.0,0.0,0.000035,0.000000
3,18.0,0.0,2.0,0.000000,0.000068
4,19.0,1.0,2.0,0.000035,0.000068
...,...,...,...,...,...
980,1392.0,0.0,1.0,0.000000,0.000034
981,1438.0,1.0,0.0,0.000035,0.000000
982,1507.0,1.0,0.0,0.000035,0.000000
983,1780.0,0.0,1.0,0.000000,0.000034


In [ ]:
fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x =bas.gfv_round, y=signal.savgol_filter(bas.pp_control,53, 3), mode="lines", name ="control (MOV 5)"), secondary_y=False)
fig.add_trace(go.Scatter(x =bas.gfv_round, y=signal.savgol_filter(bas.pp_variation1,53,3), mode="lines", name ="variation1 (MOV 8)"), secondary_y=False)
#fig.add_trace(go.Scatter(x =tt_order.tt, y=tt_order.delta, mode="lines", name ="delta", line = dict(color='gray', width=4, dash='dash')), secondary_y=True)
# Add figure title
fig.update_layout(title_text="Order across GFV of Control and Test")
# Set x-axis title
fig.update_xaxes(title_text="Gross Food Value")
# Set y-axes titles
fig.update_yaxes(title_text="Order Share", showgrid=False, secondary_y=False)
#fig.update_yaxes(title_text="Delta % difference of Variation1 vs Control", showgrid=False,secondary_y=True)
fig.show()

#### Tier level

In [ ]:
fig = px.line(df_tier, x="local_date", y="gfv_local", color = "variant", facet_col="tier", facet_col_wrap=5,
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=800, width=1600,
              category_orders = {"zone_name":['Amk', 'Bedok', 'Bukit timah', 'Bukitpanjang', 'Downtown','Far_east', 'Geylang', 'Jurong east', 'Jurongwest', 'Novena','Sengkang', 'Serangoon', 'Woodlands', 'Yishun'], 
                                 "variant": ["Control", "Variation1", "Variation2", "Variation3"]},
              title="Avg. GFV in all SG Zones across testing period")
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(showticklabels=True)

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''


# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Avg. revenue", showgrid=False)

fig.show()

In [ ]:
# create empty array list
rows_list = []

# loop the data for each zone and each stage
for zone_name in zone_names:
    zone_data = df_data[df_data["tier"] == zone_name] #filter relevant zone and stage for each loop
    zone_data = zone_data.fillna(0)
    mean_control = round(zone_data["gfv_local_control"].sum()/zone_data["order_control"].sum(),2)
    mean_variation1 = round(zone_data["gfv_local_variation1"].sum()/zone_data["order_variation1"].sum(),2)
    std_control = round(zone_data["avg_gfv_control"].std(),2)
    std_variation1 = round(zone_data["avg_gfv_variation1"].std(),2)
    cor_control_variation1 = round(zone_data["avg_gfv_control"].corr(zone_data["avg_gfv_variation1"]),2)
    sample_size_control = zone_data["order_control"].sum()
    sample_size_variation1 = zone_data["order_variation1"].sum()
    #sample_size_total = sample_size_control + sample_size_variation1
    sig_control_variation1_oneside = round(stats.wilcoxon(zone_data['avg_gfv_variation1'], zone_data['avg_gfv_control']) [1]/2,2)
    rows_list.append([zone_name, mean_control, mean_variation1, 
                      std_control, std_variation1, 
                      cor_control_variation1, 
                      sample_size_control, sample_size_variation1,  
                      sig_control_variation1_oneside])

# load data into dataframe
result = pd.DataFrame(rows_list, columns=['zone_name','mean_control','mean_variation1',
                                          'std_control', 'std_variation1', 
                                          'correlation (cV1)',  
                                          'size_control', 'size_variation1', 
                                          'p-value(v1 vs control)'])
# add two columns to data frame
#result["significance"] = result["p-value"]<=0.05
result["delta(v1 vs control)"] = round(result["mean_variation1"]/result["mean_control"]-1,2)
result.rename(columns={"zone_name":"tier"}, inplace=True)
result


,tier,mean_control,mean_variation1,std_control,std_variation1,correlation (cV1),size_control,size_variation1,p-value(v1 vs control),delta(v1 vs control)
0,Control 5 & Test 10,112.81,117.68,31.06,34.26,0.47,3862,3294,0.00,0.04
1,Control 15 & Test 15,140.85,139.82,34.72,31.38,0.68,6410,6844,0.35,-0.01
2,DF>15,151.87,154.14,33.91,34.29,0.61,8970,9360,0.24,0.01
3,Control 15 & Test 10,138.78,128.89,40.51,31.22,0.42,3179,3957,0.00,-0.07
4,Control 5 & Test: 5,106.06,104.45,25.04,24.51,0.60,6054,6038,0.46,-0.02


### Revenue/Profit

#### Aggregated Level

Average revenue

In [ ]:
fig = px.line(df_agg, x="local_date", y="rev", color = "variant", 
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=600, width=1200,
              title="Avg. Revenue across testing period")

# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Revenue", showgrid=False)

fig.show()

In [ ]:
df_sig.head()

,local_date,hour_block,delivery_fee_local_Control,delivery_fee_local_Variation1,gfv_local_Control,gfv_local_Variation1,order_Control,order_Variation1,profit_Control,profit_Variation1,rev_Control,rev_Variation1,rev_sum_Control,rev_sum_Variation1,travel_time_Control,travel_time_Variation1
0,2020-08-21,9,11.964286,11.401869,92.434524,96.327103,84,106,-20.701,200.634,34.045833,36.071963,2859.85,3859.70,4.306310,4.056729
1,2020-08-21,10,11.517857,12.222222,107.008929,101.455556,56,45,168.847,107.831,36.786964,37.629778,2060.07,1693.34,4.072500,3.992000
2,2020-08-21,11,13.689320,14.166667,124.796117,124.469298,103,114,723.858,665.904,44.764660,45.458772,4610.76,5182.30,4.595049,4.524298
3,2020-08-21,12,12.529070,13.120567,125.450581,122.170213,172,141,775.271,887.096,43.780814,44.848723,7530.30,6323.67,4.024767,4.352128
4,2020-08-21,13,12.075472,13.587786,109.193396,109.492366,106,131,200.709,873.955,40.675943,41.396107,4311.65,5422.89,4.037264,4.583588


In [ ]:
print("mean difference is",round(df_sig['rev_Variation1'].mean()/df_sig['rev_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['rev_Variation1'], df_sig['rev_Control'])[1]/2)

mean difference is 0.0064 p-value is 0.019022930603710533


2. Total revenue

In [ ]:
fig = px.line(df_agg, x="local_date", y="rev_sum", color = "variant", 
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=600, width=1200,
              title="Total revenue across testing period")

# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Revenue", showgrid=False)

fig.show()

In [ ]:
print("mean difference is",round(df_sig['rev_sum_Variation1'].mean()/df_sig['rev_sum_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['rev_sum_Variation1'], df_sig['rev_sum_Control'])[1]/2)

mean difference is 0.0429 p-value is 1.352036488244719e-06


In [14]:
df_sig.head()

,local_date,hour_block,delivery_fee_local_Control,delivery_fee_local_Variation1,gfv_local_Control,gfv_local_Variation1,order_Control,order_Variation1,profit_Control,profit_Variation1,rev_Control,rev_Variation1,rev_sum_Control,rev_sum_Variation1,travel_time_Control,travel_time_Variation1
0,2020-08-21,9,11.964286,11.401869,92.434524,96.327103,84,107,-50.660,182.471,34.045833,36.071963,2859.85,3859.70,4.306310,4.041215
1,2020-08-21,10,11.517857,12.222222,107.008929,101.455556,56,45,168.847,107.831,36.786964,37.629778,2060.07,1693.34,4.072500,3.992000
2,2020-08-21,11,13.689320,14.166667,124.796117,124.469298,103,114,714.519,672.809,44.764660,45.458772,4610.76,5182.30,4.595049,4.527632
3,2020-08-21,12,12.529070,13.120567,125.450581,122.170213,172,141,717.834,862.866,43.780814,44.848723,7530.30,6323.67,4.024767,4.349787
4,2020-08-21,13,12.075472,13.587786,109.193396,109.492366,106,131,200.709,873.955,40.675943,41.396107,4311.65,5422.89,4.037264,4.575038


In [16]:
print("mean difference is",round(df_sig['profit_Variation1'].mean()/df_sig['profit_Control'].mean()-1,4),"p-value is",stats.wilcoxon(df_sig['profit_Variation1'], df_sig['profit_Control'])[1]/2)

mean difference is 0.1795 p-value is 0.014401314572894551


#### Tier Level

1. avg revenue

In [ ]:
fig = px.line(df_tier, x="local_date", y="rev", color = "variant", facet_col="tier", facet_col_wrap=5,
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=800, width=1600,
              category_orders = {"tier":["Control 5 & Test: 5", "Control 5 & Test 10", "Control 15 & Test 10","Control 15 & Test 15", "DF>15"], 
                                 "variant": ["Control", "Variation1", "Variation2", "Variation3"]},
              title="Avg. Revenue across testing period")
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(showticklabels=True)

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''


# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Avg. Revenue", showgrid=False)

fig.show()

In [ ]:
# create empty array list
rows_list = []

# loop the data for each zone and each stage
for zone_name in zone_names:
    zone_data = df_data[df_data["tier"] == zone_name] #filter relevant zone and stage for each loop
    zone_data = zone_data.fillna(0)
    mean_control = round(zone_data["rev_control"].sum()/zone_data["order_control"].sum(),2)
    mean_variation1 = round(zone_data["rev_variation1"].sum()/zone_data["order_variation1"].sum(),2)
    std_control = round(zone_data["avg_rev_control"].std(),2)
    std_variation1 = round(zone_data["avg_rev_variation1"].std(),2)
    cor_control_variation1 = round(zone_data["avg_rev_control"].corr(zone_data["avg_rev_variation1"]),2)
    sample_size_control = zone_data["order_control"].sum()
    sample_size_variation1 = zone_data["order_variation1"].sum()
    #sample_size_total = sample_size_control + sample_size_variation1
    sig_control_variation1_oneside = round(stats.wilcoxon(zone_data['avg_rev_variation1'], zone_data['avg_rev_control']) [1]/2,2)
    rows_list.append([zone_name, mean_control, mean_variation1,
                      std_control, std_variation1, 
                      cor_control_variation1, 
                      sample_size_control, sample_size_variation1,  
                      sig_control_variation1_oneside])

# load data into dataframe
result = pd.DataFrame(rows_list, columns=['zone_name','mean_control','mean_variation1',
                                          'std_control', 'std_variation1', 
                                          'correlation (cV1)',  
                                          'size_control', 'size_variation1', 
                                          'p-value(v1 vs control)'])
# add two columns to data frame
#result["significance"] = result["p-value"]<=0.05
result["delta(v1 vs control)"] = round(result["mean_variation1"]/result["mean_control"]-1,2)
result.rename(columns={"zone_name":"tier"}, inplace=True)
result

,tier,mean_control,mean_variation1,std_control,std_variation1,correlation (cV1),size_control,size_variation1,p-value(v1 vs control),delta(v1 vs control)
0,Control 5 & Test 10,32.90,38.83,7.55,8.45,0.39,3862,3294,0.00,0.18
1,Control 15 & Test 15,49.06,48.75,8.47,7.71,0.65,6410,6844,0.28,-0.01
2,DF>15,57.90,58.44,8.95,8.97,0.60,8970,9360,0.20,0.01
3,Control 15 & Test 10,48.13,41.61,9.62,7.69,0.35,3179,3957,0.00,-0.14
4,Control 5 & Test: 5,31.96,31.73,6.01,5.77,0.58,6054,6038,0.34,-0.01


2. total revenue

In [ ]:
fig = px.line(df_tier, x="local_date", y="rev_sum", color = "variant", facet_col="tier", facet_col_wrap=5,
              #facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
              #facet_col_spacing=0.04, # default is 0.03
              height=800, width=1600,
              category_orders = {"tier":["Control 5 & Test: 5", "Control 5 & Test 10", "Control 15 & Test 10","Control 15 & Test 15", "DF>15"], 
                                 "variant": ["Control", "Variation1", "Variation2", "Variation3"]},
              title="Total Revenue across testing period")
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(showticklabels=True)

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''


# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Total Revenue", showgrid=False)

fig.show()

In [ ]:
# create empty array list
rows_list = []

# loop the data for each zone and each stage
for zone_name in zone_names:
    zone_data = df_data[df_data["tier"] == zone_name] #filter relevant zone and stage for each loop
    zone_data = zone_data.fillna(0)
    mean_control = round(zone_data["rev_control"].sum(),2)
    mean_variation1 = round(zone_data["rev_variation1"].sum(),2)
    std_control = round(zone_data["rev_control"].std(),2)
    std_variation1 = round(zone_data["rev_variation1"].std(),2)
    cor_control_variation1 = round(zone_data["rev_control"].corr(zone_data["rev_variation1"]),2)
    sample_size_control = zone_data["order_control"].sum()
    sample_size_variation1 = zone_data["order_variation1"].sum()
    #sample_size_total = sample_size_control + sample_size_variation1
    sig_control_variation1_oneside = round(stats.wilcoxon(zone_data['rev_control'], zone_data['rev_variation1']) [1]/2,2)
    rows_list.append([zone_name, mean_control, mean_variation1,
                      std_control, std_variation1, 
                      cor_control_variation1, 
                      sample_size_control, sample_size_variation1,  
                      sig_control_variation1_oneside])

# load data into dataframe
result = pd.DataFrame(rows_list, columns=['zone_name','mean_control','mean_variation1',
                                          'std_control', 'std_variation1', 
                                          'correlation (cV1)',  
                                          'size_control', 'size_variation1', 
                                          'p-value(v1 vs control)'])
# add two columns to data frame
#result["significance"] = result["p-value"]<=0.05
result["delta(v1 vs control)"] = round(result["mean_variation1"]/result["mean_control"]-1,2)
result.rename(columns={"zone_name":"tier"}, inplace=True)
result.sort_values("tier")

,tier,mean_control,mean_variation1,std_control,std_variation1,correlation (cV1),size_control,size_variation1,p-value(v1 vs control),delta(v1 vs control)
3,Control 15 & Test 10,153002.23,164660.07,346.52,342.64,0.69,3179,3957,0.00,0.08
1,Control 15 & Test 15,314446.65,333635.80,689.50,694.46,0.84,6410,6844,0.00,0.06
0,Control 5 & Test 10,127051.40,127900.76,268.30,239.90,0.67,3862,3294,0.45,0.01
4,Control 5 & Test: 5,193510.03,191593.93,360.06,304.67,0.75,6054,6038,0.42,-0.01
2,DF>15,519399.76,546993.26,1152.14,1195.08,0.91,8970,9360,0.00,0.05
